In [3]:
import pandas as pd
import pyodbc

In [40]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-M687F4C;DATABASE=AdventureWorks2014;UID=eo89;Trusted_Connection=yes')
sql = """
WITH IndivSalesData AS
(
    SELECT
       SaleMonth = MONTH(oh.OrderDate),
	   SaleYear = YEAR(oh.OrderDate),
       SalesForMonth = SUM(od.LineTotal) 
    FROM 
        AdventureWorks2014.Sales.SalesOrderDetail od 
    INNER JOIN 
        AdventureWorks2014.Sales.SalesOrderHeader oh ON od.SalesOrderID = oh.SalesOrderID
    INNER JOIN 
        AdventureWorks2014.Sales.Customer c ON oh.CustomerID = c.CustomerID
    INNER JOIN 
        AdventureWorks2014.Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE
        p.PersonType = 'IN'
    GROUP BY
       MONTH(oh.OrderDate),
	   YEAR(oh.OrderDate)
), 

CorporateSalesData AS
(
    SELECT
       SaleMonth = MONTH(oh.OrderDate),
	   SaleYear = YEAR(oh.OrderDate),
       SalesForMonth = SUM(od.LineTotal) 
    FROM 
        AdventureWorks2014.Sales.SalesOrderDetail od 
    INNER JOIN 
        AdventureWorks2014.Sales.SalesOrderHeader oh ON od.SalesOrderID = oh.SalesOrderID
    INNER JOIN 
        AdventureWorks2014.Sales.Customer c ON oh.CustomerID = c.CustomerID
    INNER JOIN 
        AdventureWorks2014.Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE
        p.PersonType = 'CS'
    GROUP BY
       MONTH(oh.OrderDate),
	   YEAR(oh.OrderDate)
)
-- Determine total sales
SELECT 
    Indiv.SaleMonth,
	Indiv.SaleYear,
    TotalSales = Indiv.SalesForMonth + CS.SalesForMonth
FROM 
    IndivSalesData Indiv
INNER JOIN 
    CorporateSalesData CS ON Indiv.SaleMonth = CS.SaleMonth AND Indiv.SaleYear = CS.SaleYear
ORDER BY 
    Indiv.SaleMonth,
	Indiv.SaleYear
;"""

data = pd.read_sql(sql,cnxn)

df = pd.DataFrame(data)
print(data)

df.to_excel (r'C:\Users\eo89\Documents\monthly_sales.xlsx', index = False, header=True)

    SaleMonth  SaleYear    TotalSales
0           1      2012  3.970627e+06
1           1      2013  2.087872e+06
2           1      2014  4.289818e+06
3           2      2012  1.475427e+06
4           2      2013  2.316922e+06
5           2      2014  1.337725e+06
6           3      2012  2.975748e+06
7           3      2013  3.412069e+06
8           3      2014  7.217531e+06
9           4      2012  1.634601e+06
10          4      2013  2.532266e+06
11          4      2014  1.797174e+06
12          5      2011  5.038059e+05
13          5      2012  3.074603e+06
14          5      2013  3.245624e+06
15          5      2014  5.366675e+06
16          6      2012  4.099354e+06
17          6      2013  5.081069e+06
18          7      2011  2.044600e+06
19          7      2012  3.417954e+06
20          7      2013  4.896354e+06
21          8      2011  2.495817e+06
22          8      2012  2.175637e+06
23          8      2013  3.333964e+06
24          9      2012  3.454152e+06
25          

In [34]:
sql = """SELECT COALESCE(addre.City, stateprov.CountryRegionCode) as Country_Region_or_City,
       Count(*) AS Total
FROM AdventureWorks2014.Person.Address addre INNER JOIN
     AdventureWorks2014.Person.StateProvince stateprov
     ON addre.StateProvinceID = stateprov.StateProvinceID
GROUP BY GROUPING SETS ((addre.City), (stateprov.CountryRegionCode))
ORDER BY Total DESC;"""
data = pd.read_sql(sql,cnxn)

df = pd.DataFrame(data)

df.to_excel (r'C:\Users\eo89\Documents\regional_sales.xlsx', index = False, header=True)

    Country_Region_or_City  Total
0                       US   8668
1                       AU   3632
2                       GB   1954
3                       FR   1851
4                       DE   1821
..                     ...    ...
576       North Sioux City      1
577               Westport      1
578                  Boise      1
579                Augusta      1
580             San Ysidro      1

[581 rows x 2 columns]


In [31]:
sql = """;WITH IndivSalesData AS
(
    SELECT
       SaleYear = YEAR(oh.OrderDate),
       SalesForYear = SUM(od.LineTotal) 
    FROM 
        AdventureWorks2014.Sales.SalesOrderDetail od 
    INNER JOIN 
        AdventureWorks2014.Sales.SalesOrderHeader oh ON od.SalesOrderID = oh.SalesOrderID
    INNER JOIN 
        AdventureWorks2014.Sales.Customer c ON oh.CustomerID = c.CustomerID
    INNER JOIN 
        AdventureWorks2014.Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE
        p.PersonType = 'IN'
    GROUP BY
       YEAR(oh.OrderDate)
), 

CorporateSalesData AS
(
    SELECT
       SaleYear = YEAR(oh.OrderDate),
       SalesForYear = SUM(od.LineTotal) 
    FROM 
        AdventureWorks2014.Sales.SalesOrderDetail od 
    INNER JOIN 
        AdventureWorks2014.Sales.SalesOrderHeader oh ON od.SalesOrderID = oh.SalesOrderID
    INNER JOIN 
        AdventureWorks2014.Sales.Customer c ON oh.CustomerID = c.CustomerID
    INNER JOIN 
        AdventureWorks2014.Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE
        p.PersonType = 'CS'
    GROUP BY
       YEAR(oh.OrderDate)
)
SELECT 
    Indiv.SaleYear, 
    TotalSales = Indiv.SalesForYear + CS.SalesForYear
FROM 
    IndivSalesData Indiv
INNER JOIN 
    CorporateSalesData CS ON Indiv.SaleYear = CS.SaleYear
ORDER BY 
    Indiv.SaleYear"""
data = pd.read_sql(sql,cnxn)

df = pd.DataFrame(data)

df.to_excel (r'C:\Users\eo89\Documents\export_dataframe.xlsx', index = False, header=True)

   SaleYear  PercentToIndividuals  PercentToCorporate    TotalSales
0      2011               30.5586             69.4413  1.264167e+07
1      2012               19.0625             80.9374  3.352430e+07
2      2013               24.6022             75.3977  4.362248e+07
3      2014               41.7432             58.2567  2.005793e+07
